## Transfer Learning Clusterization

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.utils import to_categorical
from keras.models import load_model, Model

In [3]:
model = load_model(filepath='ResNet164.h5')

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255                         
x_test /= 255

In [5]:
n_clusters = 10

train_indices = np.row_stack([np.argwhere(y_train == 0), np.argwhere(y_train == 1)])
train_indices = train_indices.reshape(train_indices.shape[0],)
np.random.shuffle(train_indices)

test_indices = np.row_stack([np.argwhere(y_test == 0), np.argwhere(y_test == 1)])
test_indices = test_indices.reshape(test_indices.shape[0],)
np.random.shuffle(test_indices)

x_train = x_train[train_indices]
y_train = y_train[train_indices]
x_test = x_test[test_indices]
y_test = y_test[test_indices]

y_train = to_categorical(y_train, n_clusters)
y_test = to_categorical(y_test, n_clusters)

x_train = x_train.reshape((x_train.shape[0],28,28,1))
x_test = x_test.reshape((x_test.shape[0],28,28,1))

In [6]:
#for choose layer
l = model.layers
for i in range(len(l)):
    print(l[i].name)

input_1
zero_padding2d_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
conv2d_5
conv2d_4
add_1
batch_normalization_4
activation_4
conv2d_6
batch_normalization_5
activation_5
conv2d_7
batch_normalization_6
activation_6
conv2d_8
add_2
batch_normalization_7
activation_7
conv2d_9
batch_normalization_8
activation_8
conv2d_10
batch_normalization_9
activation_9
conv2d_11
add_3
batch_normalization_10
activation_10
conv2d_12
batch_normalization_11
activation_11
conv2d_13
batch_normalization_12
activation_12
conv2d_14
add_4
batch_normalization_13
activation_13
conv2d_15
batch_normalization_14
activation_14
conv2d_16
batch_normalization_15
activation_15
conv2d_17
add_5
batch_normalization_16
activation_16
conv2d_18
batch_normalization_17
activation_17
conv2d_19
batch_normalization_18
activation_18
conv2d_20
add_6
batch_normalization_19
activation_19
conv2d_21
batch_normalization_20
activation_20
conv2d_22
batch_

In [7]:
layer = model.get_layer(l[-2].name)
layer_model = Model(inputs=model.input, outputs=layer.output)

output = layer_model.predict(x_test)

In [8]:
output.shape

(2115, 256)

In [9]:
Accuracies = {}
label = np.argmax(y_test, axis=1)
n = y_test.shape[0]

In [10]:
from sklearn import cluster

In [11]:
#KMeans
KMeans_predictions = cluster.KMeans(n_clusters=2, random_state=4).fit_predict(output)
Accuracies['KMeans'] = (n - np.count_nonzero(label - KMeans_predictions)) / n

#Affinity Propagation
AffinityPropagation_predictions = cluster.AffinityPropagation(preference=-5, damping = 0.95).fit_predict(output)
Accuracies['AffinityPropagation'] = (n - np.count_nonzero(label - AffinityPropagation_predictions)) / n

#Mean Shift
MeanShift_predictions = cluster.MeanShift().fit_predict(output)
Accuracies['MeanShift'] = (n - np.count_nonzero(label - MeanShift_predictions)) / n

#Spectral Clustering
SpectralClustering_predictions = cluster.SpectralClustering(n_clusters=2).fit_predict(output)
Accuracies['SpectralClustering'] = (n - np.count_nonzero(label - SpectralClustering_predictions)) / n

#Agglomerative Clustering
AgglomerativeClustering_predictions = cluster.AgglomerativeClustering(n_clusters=2).fit_predict(output)
Accuracies['AgglomerativeClustering'] = (n - np.count_nonzero(label - AgglomerativeClustering_predictions)) / n

#DBSCAN
DBSCAN_predictions = cluster.DBSCAN(eps=10).fit_predict(output)
Accuracies['DBSCAN'] = (n - np.count_nonzero(label - DBSCAN_predictions)) / n

In [16]:
sorted(Accuracies.items(), key=lambda x: x[1])

[('AgglomerativeClustering', 0.00047281323877068556),
 ('AffinityPropagation', 0.018912529550827423),
 ('DBSCAN', 0.46335697399527187),
 ('MeanShift', 0.9470449172576832),
 ('KMeans', 0.9583924349881797),
 ('SpectralClustering', 0.966903073286052)]